In [ ]:
import os
import re
import pandas as pd
from PIL import Image
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from torch.cuda.amp import GradScaler, autocast

: 

In [ ]:
# Define models
models_dict = {
    "MobileNetV3S": models.mobilenet_v3_small(weights=None),
    "ShuffleNetV2": models.shufflenet_v2_x1_0(weights=None),
    "MobileNetV2": models.mobilenet_v2(weights=None),
    "ResNet18": models.resnet18(weights=None),
    "SqueezeNet": models.squeezenet1_0(weights=None),
}


In [ ]:
# Training and evaluation
dataset_path = "./dataset"
augment_data = False
num_epochs = 100
batch_size = 128
learning_rate = 0.001
criterion = nn.MSELoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Define the augmentations
augmentations = {
    "brightness": transforms.Compose(
        [
            transforms.ColorJitter(brightness=0.5),
            transforms.ToTensor(),
            transforms.ToPILImage(),
        ]
    ),
    "contrast": transforms.Compose(
        [
            transforms.ColorJitter(contrast=0.5),
            transforms.ToTensor(),
            transforms.ToPILImage(),
        ]
    ),
    "saturation": transforms.Compose(
        [
            transforms.ColorJitter(saturation=0.5),
            transforms.ToTensor(),
            transforms.ToPILImage(),
        ]
    ),
    "hue": transforms.Compose(
        [
            transforms.ColorJitter(hue=0.5),
            transforms.ToTensor(),
            transforms.ToPILImage(),
        ]
    ),
}


In [ ]:
dirs = [
    d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))
]

if augment_data:
    for dir in dirs:
        files = os.listdir(os.path.join(dataset_path, dir))

        for file_name in tqdm(files, desc=f"Processing {dir}"):
            file_path = os.path.join(dataset_path, dir, file_name)
            photo_id = re.search(r"\d+", file_name).group()

            if file_name != f"{photo_id}.jpg":
                continue

            image = Image.open(file_path).convert("RGB")

            for name, augmentation in augmentations.items():
                augmented_image = augmentation(image)
                augmented_image = np.array(augmented_image)
                augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
                save_path = os.path.join(dataset_path, dir, f"{photo_id}_{name}.jpg")
                cv2.imwrite(save_path, augmented_image)

#

In [ ]:
# Custom Dataset Class
class JetBotDataset(Dataset):
    def __init__(self, target_map, transform=None):
        self.target_map = target_map
        self.transform = transform

    def __len__(self):
        return len(self.target_map)

    def __getitem__(self, idx):
        img_path, target = self.target_map[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        target = torch.tensor(target, dtype=torch.float32)
        return image, target

In [ ]:
def get_target(path, dirs):
    result = []
    for dir in dirs:
        labels = pd.read_csv(
            f"{path}/{dir}.csv", header=None, index_col=0, names=["speed", "turn"]
        )
        for file_name in os.listdir(os.path.join(path, dir)):
            file_path = os.path.join(path, dir, file_name)
            photo_id = int(re.search(r"\d+", file_name).group())
            if photo_id in labels.index:
                target = labels.loc[photo_id]
                result.append((file_path, (target["speed"], target["turn"])))
    return result

In [ ]:
target_map = get_target(dataset_path, dirs)
train_data, test_data = train_test_split(target_map, test_size=0.2, random_state=42)

transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

train_dataset = JetBotDataset(train_data, transform=transform)
test_dataset = JetBotDataset(test_data, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)